# Create Coastline Segments

This notebook modifies the CoDEC points to generate a uniform set of coastline segment centroids.

**TODO**

- [ ] add back in the last section (with filepaths appropriately in settings.py) when we figure out where `ne_coastline_polys_CIAM_exposure_matches.parquet` is generated

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import Point

from sliiders.settings import (
    DIR_GTSM_STATIONS_TOTHIN,
    PATH_SEG_CENTROIDS,
    PATH_GTSM_SURGE,
)

In [ ]:
GTSMPTS_EUR = DIR_GTSM_STATIONS_TOTHIN / "gtsm_stations_eur_tothin.shp"

### Import GTSM Station Points & Thin Europe

In [ ]:
# Import GTSM (CoDEC) stations present in ERA5 GTSM data from Muis et al. 2020
gtsm0 = (
    xr.open_dataset(PATH_GTSM_SURGE)
    .rename(station_y_coordinate="lat", station_x_coordinate="lon")
    .station_id.drop("station_name")
    .reset_coords()
)
gtsm0["station_id"] = gtsm0.station_id.astype(str)
gtsm0 = gtsm0.to_dataframe()

# Import Europe stations to be thinned (came in 5x higher res than rest of world)
# Stations to be thinned were manually ID'ed in GIS
gtsm_e0 = gpd.read_file(GTSMPTS_EUR)
gtsm_e0["serial_id"] = [gtsm_e0.station_id[i][-5:] for i in range(len(gtsm_e0))]
gtsm_e0 = gtsm_e0.sort_values("serial_id")

# Filter 80% of Europe stations
gtsm_e1 = gtsm_e0.iloc[np.arange(0, len(gtsm_e0), 5)]

# Update full GTSM layer
gtsm1 = gtsm0[
    ~gtsm0.station_id.isin(gtsm_e0.station_id)
]  # all stations not in the to-be-thinned orig Europe set
gtsm1 = pd.concat([gtsm1, gtsm_e1])

In [ ]:
# Add back in Gibraltar and Monaco
gib_id = "id_coast_glob_eur_03452"
mon_id = "id_coast_glob_eur_03236"
gtsm1 = pd.concat([gtsm1, gtsm0[gtsm0.station_id.isin([gib_id, mon_id])]])

In [ ]:
# Add new manual points for 15 small country coastlines
lats = [
    18.20427252,  # Anguilla
    42.92257646,  # Bosnia & Herzegovina
    17.92105954,  # St Barthelemy
    -54.45126484,  # Bouvet Island
    -12.1888075,  # Cocos (Keeling) Islands
    10.28952433,  # Clipperton Island
    29.51144515,  # Jordan
    16.69068301,  # Montserrat
    -29.01453206,  # Norfolk Island
    -0.54073669,  # Nauru
    -24.38660564,  # Pitcairn Islands
    10.73177001,  # Spratly Islands
    -9.3415246,  # Tokelau
    19.28118539,  # US Minor Outlying Island (Wake Island)
    18.03885916,
]  # Sint Maarten

lons = [
    -63.05668448,
    17.61671166,
    -62.82955182,
    3.35020284,
    96.83802356,
    -109.21026241,
    34.97905326,
    -62.18841426,
    167.97463688,
    166.91406099,
    -128.32974227,
    115.8022823,
    -171.19264163,
    166.64951319,
    -63.01482338,
]

add_pts = {
    "station_id": ["id_coast_glob_990" + str(i + 1).zfill(2) for i in range(len(lats))],
    "lat": lats,
    "lon": lons,
    "geometry": [Point(lons[i], lats[i]) for i in range(len(lats))],
}

add_pts = gpd.GeoDataFrame(add_pts, crs="EPSG:4326")

gtsm1 = pd.concat([gtsm1, add_pts])

In [ ]:
# Manual deletions of certain points that don't fall nearer to coastlines than other points
del_ids = [
    "eur_01019",
    "eur_01812",
    "eur_00979",
    "13536",
    "14447",
    "15646",
    "18265",
    "18656",
    "18720",
    "18724",
]
del_ids = ["id_coast_glob_" + del_ids[i] for i in range(len(del_ids))]
gtsm1 = gtsm1[~gtsm1.station_id.isin(del_ids)]

In [ ]:
# Remove buoy and obs points
gtsm1 = gtsm1[~gtsm1.station_id.str.contains("buoy")]
gtsm1 = gtsm1[~gtsm1.station_id.str.contains("obs")]

# Remove duplicates
idx_unique = pd.DataFrame(gtsm1.geometry).drop_duplicates().index
gtsm1 = gtsm1[gtsm1.index.isin(idx_unique)]

# Remove unwanted columns
gtsm1 = gtsm1.drop(columns=["fid", "field_1", "serial_id"])

In [ ]:
# Export resulting layers
gtsm1.to_file(PATH_SEG_CENTROIDS)  # final set of majority GTSM points for CIAM segs

### [ASIDE] Identify which NaturalEarth land masses have exposure (pop or capital)

In [ ]:
# land = gpd.read_parquet(TMPDIR + "ne_coastline_polys_CIAM_exposure_matches.parquet")
# land_exp = land[land.contains_exposure == True]
# land_exp.to_file(TMPDIR + "ne_coastline_polys_CIAM_wexp.shp")

# land_noexp = land[land.contains_exposure == False]
# land_noexp.to_file(TMPDIR + "ne_coastline_polys_CIAM_noexp.shp")